In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import adaptive_latents as al
# from nlb_tools.nwb_interface import NWBDataset
from pynwb import NWBHDF5IO
import pandas as pd


rng = np.random.default_rng()

In [ ]:
ls /home/jgould/Documents/Bubblewrap/generated/datasets/000128/sub-Jenkins/

In [ ]:
fpath

In [ ]:
fpath = al.CONFIG["data_path"] / '000128'/ 'sub-Jenkins'/ 'sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb'

nwb_in = NWBHDF5IO(fpath, "r").read()
nwb_in

In [ ]:
trials = nwb_in.intervals['trials'].to_dataframe()
trial_types = trials["trial_type"].unique()

In [ ]:
hand_pos = np.array(nwb_in.processing['behavior'].data_interfaces['hand_pos'].data)
hand_t = np.array(nwb_in.processing['behavior'].data_interfaces['hand_pos'].timestamps)

In [ ]:
selected_set = trials[(trials.trial_type == trial_types[0]) & (trials.trial_version == 0)]

In [ ]:
to_plot = []
for index, row in selected_set.iterrows():
    to_plot.append(hand_pos[(row.start_time < hand_t) & (hand_t < row.stop_time),:])
for reach in to_plot:
    plt.plot(reach[:,0], reach[:,1])

In [ ]:
units = nwb_in.units.to_dataframe()

In [ ]:
units

In [ ]:
bin_width = 0.03
bin_edges = np.arange(units.iloc[0,2][0,0], units.iloc[0,2][-1,-1] + bin_width, bin_width)

A = np.zeros((len(bin_edges)-1, units.shape[0]))

for i in range(units.shape[0]):
    A[:,i], _ = np.histogram(units.iloc[i,1], bin_edges)

recorded_intervals = units.iloc[0,2]

covers = []

interval_to_start_from = 0
# set_trace()

def intersection(start1, stop1, start2, stop2):
    return max(min(stop1, stop2) - max(start1, start2),0)

for i in range(len(bin_edges)-1):
    bin_start = bin_edges[i]
    bin_stop = bin_edges[i+1]
    covered = 0
    for j in range(interval_to_start_from, recorded_intervals.shape[0]):
        interval_start, interval_stop = recorded_intervals[j]
        if bin_start > interval_stop:
            interval_to_start_from += 1
            continue
        if interval_start > bin_stop:
            break
        covered +=  intersection(bin_start, bin_stop, interval_start, interval_stop)
        
    if covered/bin_width < .9:
        A[i,:] = np.nan

In [ ]:
plt.imshow(A.T, aspect='auto', interpolation='nearest')

In [ ]:
hand_pos.shape

In [ ]:
import adaptive_latents

In [ ]:

bin_centers = np.convolve(bin_edges,[.5, .5],'valid')
resampled_hand = adaptive_latents.input_sources.functional.resample_behavior(hand_pos, bin_centers, hand_t)

In [ ]:
np.column_stack([A, resampled_hand])

In [ ]:
%matplotlib inline
for i in range(10):
    times = units[~units.heldout].iloc[i,1]
    plt.plot(times, times*0+i, '.', color='C0')
    # for times in units[~units.heldout].iloc[i,2]:
    #     plt.plot(times, times*0+i, '-', color='red', alpha=.1)

In [ ]:
nwbfile

In [ ]:
trials

In [ ]:
ls /home/jgould/Documents/Bubblewrap/generated/datasets/000128/sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb